In [1]:
import pandas as pd

In [ ]:
crime_data_df = pd.read_csv("../data/crime.csv")
crime_data_df.describe

crime_data_sample_df = pd.read_csv("../data/crime.csv").sample(500)
crime_data_sample_df

offense_codes_df = pd.read_csv("../data/offense_codes.csv")